In [1]:
from sqlalchemy import create_engine,Table, Column, Integer, String, MetaData
import requests
import pandas as pd
from bs4 import BeautifulSoup
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

In [2]:
engine= create_engine('sqlite:///Swiggy.db',echo = False)
meta = MetaData()

In [3]:
from sqlalchemy import ForeignKey

In [3]:
Restaurant = Table(
   'Restaurant', meta, 
   Column('Id', Integer, primary_key = True),
   Column('name', String),
   Column('Link',String), 
   Column('Speciality',String),
   Column('Pure-Veg',String),
   Column('CostFor2',Integer),
   Column('Location',String), 
   Column('extracted',String),extend_existing=True
)

In [ ]:
Menu = Table(
   'Menu', meta,
   Column('RestID', Integer,ForeignKey('Restaurant.Id'), nullable=False), 
   Column('Rest_name', String),
   Column('Food_id',Integer), 
   Column('Food_name',String),
   Column('Food_type',String), 
   Column('Price',Integer) 
#ForeignKeyConstraint(['invoice_id', 'ref_num'], ['invoice.invoice_id', 'invoice.ref_num'])
)

In [15]:
 engine.execute('ALTER TABLE %s ADD COLUMN %s %s' % (Menu, "Food_type", "VARCHAR"))

In [5]:
meta.create_all(engine)

In [4]:
from sqlalchemy.orm import sessionmaker

In [5]:
from sqlalchemy.ext.declarative import declarative_base

In [6]:
Session=sessionmaker(bind=engine)
session=Session()

In [7]:
def extractData(rest_id,url):
    try:
        query = url
        page = requests.get(query, headers)
        soup = BeautifulSoup(page.text, 'html.parser')

        restuarant = soup.find('div', class_='_1637z')
        rating=restuarant.find('div', class_='_2l3H5').text 

        string= restuarant.find_all('div', class_='_2l3H5')[2]
        cost=string.find('span')
        if cost!=None:
            cos=cost.text[2:]
        else:
            cos=None
        #costlist.append(cost)
        #print(cost)
        try:
            if(restuarant.find('span', class_='_1KYwd').text=='Pure Veg'):
                #print("Pure Veg")
                #restype.append("Pure Veg")
                pveg='Pure Veg'
            else:
                #restype.append("--")
                pveg='--'
        except:
            #print("--")
            restype.append("--")
        restName = restuarant.find('h1', class_='_3aqeL').text
        #restuarantName.append(restName)
        #rest_ids_hotels.append(rest_id)
        speciality=restuarant.find('div', class_='_3Plw0 JMACF').text
        speciality=speciality.map(lambda x: x.lstrip('+-Opens next at 123456789101112:0102030405607809101520253040455060 ampm, tomorrowtoday'))
        #specialities.append(speciality)
        location= restuarant.find('div', class_='Gf2NS _2Y6HW').text
        #locations.append(location)
        #links.append(url)
        #print(rating)
        #print(restName,  speciality, location)
        rest= Restaurant(Id=no,name=restName,Link=url,Speciality=speciality,Pure_veg=pveg,CostFor2=cos,Location=location,extracted='yes')
        session.add(rest)
        li=session.query(Links).filter(Links.link==url)
        Links.extracted='yes'
        session.commit()
        menu=soup.find_all('div',class_='_2wg_t')
        for item in menu:
            #print(item.prettify()) 
            vg=item.find('i')
            #print(vg.get('class'))
            if(vg.get('class')[2]=='icon-Veg'):
                vnv='Veg'
            elif(vg.get('class')[2]=='icon-NonVeg'):                
                vnv='Non-Veg'

            itemname=item.find('h3', class_='styles_itemNameText__3bcKX').text
            
            itemprice=item.find('span', class_='rupee').text
            
            men= Menu(RestID=no,Rest_name=restName,Food_name=itemname, Food_type=vnv,Price=itemprice)
            session.add(men)
        
            
    except:
        print("page not found")
    

In [10]:
link=session.query(Links).all()
r_id=1
for li in link:
    if li[extarcted]=='no':
        extractData(r_id,li.link )
        r_id+=1

NameError: name 'Links' is not defined